# Prediccion del Campeon de Rugby del Mundo teniendo en cuenta el score de la IRB pero ademas segun los resultados de la primer zona y pesos personalizados para ajustar mas el posible Campeon.

In [251]:
import pandas as pd

In [252]:
data_paises = pd.read_csv("../data/paises.csv")
data_partidos = pd.read_csv("../data/partidos.csv")

In [253]:
data_paises

,zona,pais,score
0,A,France,89.22
1,A,Italy,75.63
2,A,Namibia,61.61
3,A,Uruguay,66.63
4,A,New Zeland,89.06
5,B,Irland,91.82
6,B,Romania,64.56
7,B,Scotland,84.01
8,B,South Africa,91.08
9,B,Tonga,70.29


In [254]:
data_partidos

,ID,equipo_local,equipo_visitante,puntos_local,puntos_visitante,fecha,zona
0,1,France,New Zeland,23,13,08/10/2023,A
1,2,Italy,Namibia,52,8,09/10/2023,A
2,3,Irland,Romania,82,8,09/10/2023,B
3,4,Australia,Georgia,35,15,09/10/2023,C
4,5,England,Argentina,27,10,09/10/2023,D


In [255]:
# paises historicos
paises_historicos = [
    "New Zeland", "South Africa", "France", "Australia", "England"
]

max_score = data_paises['score'].max()
min_score = data_paises['score'].min()

threshold = (max_score - min_score) * 0.05

In [256]:
print(f"max: {max_score}; min: {min_score}; threshold: {round(threshold, 2)}")

max: 91.82; min: 60.49; threshold: 1.57


In [257]:
def calcular_puntos(resultado, diferencia_puntos, es_equipo_historico, diferencia_score_oponente, score_irb):
    puntos = 0
    
    # Añadir puntos según el resultado
    if resultado == 'ganado':
        puntos += 2
        if diferencia_puntos > 12:
            puntos += 1
    elif resultado == 'perdido':
        puntos -= 1
    
    # Añadir puntos si es un equipo histórico
    if es_equipo_historico:
        puntos += 1
            
    return puntos



In [258]:
def es_equipo_historico(equipo):
    return equipo['pais'] in paises_historicos


In [259]:
for index, row in data_partidos.iterrows():
    equipo_local = row['equipo_local']
    equipo_visitante = row['equipo_visitante']

    print(f"Procesando el partido: {equipo_local} vs. {equipo_visitante}")

    diferencia_puntos = row['puntos_local'] - row['puntos_visitante']
    print(f"Diferencia de puntos: {diferencia_puntos}")

    if diferencia_puntos > 0:
        resultado_local = 'ganado'
        resultado_visitante = 'perdido'
    elif diferencia_puntos < 0:
        resultado_local = 'perdido'
        resultado_visitante = 'ganado'
    else:
        resultado_local = 'empatado'
        resultado_visitante = 'empatado'

    es_equipo_historico_local = equipo_local in paises_historicos
    es_equipo_historico_visitante = equipo_visitante in paises_historicos
    print(f"Es {equipo_local} un equipo histórico? {es_equipo_historico_local}")
    print(f"Es {equipo_visitante} un equipo histórico? {es_equipo_historico_visitante}")

    score_irb_local = data_paises[data_paises['pais'] == equipo_local]['score'].iloc[0]
    score_irb_visitante = data_paises[data_paises['pais'] == equipo_visitante]['score'].iloc[0]

    puntos_local = calcular_puntos(resultado_local, diferencia_puntos, es_equipo_historico_local, score_irb_local - score_irb_visitante, score_irb_local)
    puntos_visitante = calcular_puntos(resultado_visitante, -diferencia_puntos, es_equipo_historico_visitante, score_irb_visitante - score_irb_local, score_irb_visitante)

    # Definir desgaste basado en la diferencia de scores y no en el resultado del partido.
    desgaste_local = 0.5 if abs(score_irb_local - score_irb_visitante) < threshold else 0
    desgaste_visitante = 0.5 if abs(score_irb_visitante - score_irb_local) < threshold else 0

    indice_local = data_paises[data_paises['pais'] == equipo_local].index[0]
    score_anterior_local = data_paises.at[indice_local, 'score']
    nuevo_score_local = score_anterior_local + puntos_local - desgaste_local
    data_paises.at[indice_local, 'score'] = nuevo_score_local

    indice_visitante = data_paises[data_paises['pais'] == equipo_visitante].index[0]
    score_anterior_visitante = data_paises.at[indice_visitante, 'score']
    nuevo_score_visitante = score_anterior_visitante + puntos_visitante - desgaste_visitante
    data_paises.at[indice_visitante, 'score'] = nuevo_score_visitante

    print(f"Score de {equipo_local} antes de actualizar: {score_anterior_local}")
    print(f"Score de {equipo_visitante} antes de actualizar: {score_anterior_visitante}")
    print(f"Score de {equipo_local} después de actualizar: {nuevo_score_local}")
    print(f"Score de {equipo_visitante} después de actualizar: {nuevo_score_visitante}")

equipo_ganador = data_paises.sort_values(by='score', ascending=False).iloc[0]


Procesando el partido: France vs. New Zeland
Diferencia de puntos: 10
Es France un equipo histórico? True
Es New Zeland un equipo histórico? True
Score de France antes de actualizar: 89.22
Score de New Zeland antes de actualizar: 89.06
Score de France después de actualizar: 91.72
Score de New Zeland después de actualizar: 88.56
Procesando el partido: Italy vs. Namibia
Diferencia de puntos: 44
Es Italy un equipo histórico? False
Es Namibia un equipo histórico? False
Score de Italy antes de actualizar: 75.63
Score de Namibia antes de actualizar: 61.61
Score de Italy después de actualizar: 78.63
Score de Namibia después de actualizar: 60.61
Procesando el partido: Irland vs. Romania
Diferencia de puntos: 74
Es Irland un equipo histórico? False
Es Romania un equipo histórico? False
Score de Irland antes de actualizar: 91.82
Score de Romania antes de actualizar: 64.56
Score de Irland después de actualizar: 94.82
Score de Romania después de actualizar: 63.56
Procesando el partido: Australia v

In [260]:
print(f"El Equipo predicto para ganar es: {equipo_ganador['pais']}")

El Equipo predicto para ganar es: Irland


In [261]:
# Ordenar los equipos por su score final de manera descendente
equipos_ordenados = data_paises.sort_values(by='score', ascending=False).reset_index(drop=True)

# Imprimir la lista de equipos en orden
print("\nClasificación final:")
for index, equipo in equipos_ordenados.iterrows():
    print(f"{index + 1}. {equipo['pais']}: {equipo['score']}")


Clasificación final:
1. Irland: 94.82
2. France: 91.72
3. South Africa: 91.08
4. New Zeland: 88.56
5. Scotland: 84.01
6. Australia: 83.87
7. England: 83.45
8. Fiji: 80.28
9. Argentina: 79.36
10. Italy: 78.63
11. Wales: 78.26
12. Samoa: 76.19
13. Georgia: 75.23
14. Japon: 73.29
15. Tonga: 70.29
16. Portugal: 68.61
17. Uruguay: 66.63
18. Romania: 63.56
19. Namibia: 60.61
20. Chile: 60.49


In [262]:
mejores_por_grupo = data_paises.sort_values(by=["zona", "score"], ascending=[True, False]).groupby("zona").head(2)
print(mejores_por_grupo)


   zona          pais  score
0     A        France  91.72
4     A    New Zeland  88.56
5     B        Irland  94.82
8     B  South Africa  91.08
10    C     Australia  83.87
11    C          Fiji  80.28
17    D       England  83.45
15    D     Argentina  79.36


In [263]:
cuartos = [
    (mejores_por_grupo[mejores_por_grupo["zona"] == "C"].iloc[0], mejores_por_grupo[mejores_por_grupo["zona"] == "D"].iloc[1]),
    (mejores_por_grupo[mejores_por_grupo["zona"] == "B"].iloc[0], mejores_por_grupo[mejores_por_grupo["zona"] == "A"].iloc[1]),
    (mejores_por_grupo[mejores_por_grupo["zona"] == "D"].iloc[0], mejores_por_grupo[mejores_por_grupo["zona"] == "C"].iloc[1]),
    (mejores_por_grupo[mejores_por_grupo["zona"] == "A"].iloc[0], mejores_por_grupo[mejores_por_grupo["zona"] == "B"].iloc[1]),
]

In [264]:
def simular_partido(equipo1, equipo2):
    if equipo1["score"] > equipo2["score"]:
        return equipo1
    else:
        return equipo2
    
ganadores_cuartos = []

for partido in cuartos:
    equipo1, equipo2 = partido
    
    # Determina la diferencia de puntos (score)
    diferencia_puntos_equipo1 = equipo1["score"] - equipo2["score"]
    diferencia_puntos_equipo2 = -diferencia_puntos_equipo1  # Es la diferencia inversa para el segundo equipo
    
    # Determina el resultado
    if diferencia_puntos_equipo1 > 0:
        resultado_equipo1 = 'ganado'
        resultado_equipo2 = 'perdido'
    else:
        resultado_equipo1 = 'perdido'
        resultado_equipo2 = 'ganado'
    
    # Determina si el equipo es histórico
    es_historico_equipo1 = es_equipo_historico(equipo1)
    es_historico_equipo2 = es_equipo_historico(equipo2)
    
    # Calcular puntos adicionales
    puntos_adicionales_equipo1 = calcular_puntos(resultado_equipo1, diferencia_puntos_equipo1, es_historico_equipo1, diferencia_puntos_equipo1, equipo1["score"])
    puntos_adicionales_equipo2 = calcular_puntos(resultado_equipo2, diferencia_puntos_equipo2, es_historico_equipo2, diferencia_puntos_equipo2, equipo2["score"])
    
    # Modificar scores
    equipo1["score"] += puntos_adicionales_equipo1
    equipo2["score"] += puntos_adicionales_equipo2
    
    # Simular partido
    ganador = simular_partido(equipo1, equipo2)
    ganadores_cuartos.append(ganador)

In [265]:
print("Cuartos de Final:")
print("-" * 30)
for partido in cuartos:
    equipo1, equipo2 = partido
    
    # Guardar el score antes del partido
    score_anterior_equipo1 = equipo1["score"]
    score_anterior_equipo2 = equipo2["score"]
    
    # Simular el partido y actualizar scores
    ganador = simular_partido(equipo1, equipo2)
    ganadores_cuartos.append(ganador)
    
    # Obtener el score actualizado después del partido
    score_actualizado_equipo1 = data_paises[data_paises['pais'] == equipo1['pais']]['score'].iloc[0]
    score_actualizado_equipo2 = data_paises[data_paises['pais'] == equipo2['pais']]['score'].iloc[0]
    
    # Determinar el orden de impresión basado en el ganador
    if ganador['pais'] == equipo1['pais']:
        print(f"{equipo1['pais']} ({score_actualizado_equipo1}) vs {equipo2['pais']} ({score_actualizado_equipo2}) => Ganador: {ganador['pais']} ({ganador['score']})")
    else:
        print(f"{equipo2['pais']} ({score_actualizado_equipo2}) vs {equipo1['pais']} ({score_actualizado_equipo1}) => Ganador: {ganador['pais']} ({ganador['score']})")


Cuartos de Final:
------------------------------
Australia (83.87) vs Argentina (79.36) => Ganador: Australia (86.87)
Irland (94.82) vs New Zeland (88.56) => Ganador: Irland (96.82)
England (83.45) vs Fiji (80.28) => Ganador: England (86.45)
France (91.72) vs South Africa (91.08) => Ganador: France (94.72)


In [266]:
def imprimir_partido(partido, ganador):
    equipo1, equipo2 = partido
    # Actualizar el score de los equipos después del partido
    score_actualizado_equipo1 = data_paises[data_paises['pais'] == equipo1['pais']]['score'].iloc[0]
    score_actualizado_equipo2 = data_paises[data_paises['pais'] == equipo2['pais']]['score'].iloc[0]
    
    # Determinar el orden de impresión basado en el ganador
    if ganador['pais'] == equipo1['pais']:
        print(f"{equipo1['pais']} ({score_actualizado_equipo1}) vs {equipo2['pais']} ({score_actualizado_equipo2}) => Ganador: {ganador['pais']} ({ganador['score']})")
    else:
        print(f"{equipo2['pais']} ({score_actualizado_equipo2}) vs {equipo1['pais']} ({score_actualizado_equipo1}) => Ganador: {ganador['pais']} ({ganador['score']})")

print("Semifinales:")
print("-" * 30)
for partido in semifinales:
    ganador = simular_partido(*partido)
    ganadores_semifinales.append(ganador)
    imprimir_partido(partido, ganador)

Semifinales:
------------------------------
Irland (94.82) vs Australia (83.87) => Ganador: Irland (98.82)
France (91.72) vs England (83.45) => Ganador: France (97.72)


In [269]:
def imprimir_partido(partido, ganador):
    equipo1, equipo2 = partido
    if ganador['pais'] == equipo1['pais']:
        equipo_perdedor = equipo2
    else:
        equipo_perdedor = equipo1
        equipo1, equipo2 = equipo2, equipo1  # Intercambiamos los equipos para que el ganador siempre se muestre primero

    print(f"{equipo1['pais']} ({equipo1['score']}) vs {equipo2['pais']} ({equipo2['score']}) => Ganador: {ganador['pais']} ({ganador['score']})")

print("Tercer Puesto:")
print("-" * 30)
ganador_tercer_puesto = simular_partido(*tercer_puesto)
imprimir_partido(tercer_puesto, ganador_tercer_puesto)

# Actualizamos la información de los equipos de la final
final = (
    data_paises[data_paises['pais'] == final[0]['pais']].iloc[0].to_dict(),
    data_paises[data_paises['pais'] == final[1]['pais']].iloc[0].to_dict()
)

print("\nFinal:")
print("-" * 30)
campeon = simular_partido(*final)
imprimir_partido(final, campeon)


Tercer Puesto:
------------------------------
Australia (86.87) vs England (86.45) => Ganador: Australia (86.87)

Final:
------------------------------
Irland (94.82) vs France (91.72) => Ganador: Irland (94.82)
